# PYSCF Lab 1 - Mole_HF_DFT

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
1. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
March 2023

$$
\newcommand{\HH}{\mathtt{H}}  
\newcommand{\ad}{a^\dagger}  
$$

## PySCF

![wpySCF_Logo](./Graphics/Pyscf.png)

**Python-based Simulations of Chemistry Framework (PySCF)** est un programme de chimie computationnelle *ab initio* implémenté nativement dans le langage de programme Python. Le package vise à fournir une plate-forme simple, légère et efficace pour le développement et le calcul de codes de chimie quantique. Il fournit diverses fonctions pour faire la théorie de Hartree-Fock, MP2, la théorie fonctionnelle de la densité (DFT), MCSCF, la théorie des clusters couplés au niveau non relativiste et la théorie relativiste Hartree-Fock à 4 composants. PySCF est utilisé dans la plupart des packages de calculs quantiques pour les calculs HF (première quantification) et diverses étapes des transformations de seconde quantification.

Il est à noter que la méthode SCF ou Self Consistent Field est une méthode variationnelle où la fonction d'essai est un déterminant de Slater ou une combinaison lineaire sw déterminants.

Dans cette série de tutoriels, nous examinons l'utilisation de quelques modules usuels. Pour plus de détails, voir le site de [pyscf](https://pyscf.org/user/scf.html).

### Creating a molecule (mol) object with `gto.Mole()`

Considérons le dimère d'eau (deux molécules d'eau à liaison hydrogène) illustré par la figure suivante

![water_dimer.jpg](./Graphics/water_dimer.jpg)



In [1]:
from pyscf import gto # Gaussian type orbitals

In [2]:
DWat_mol=gto.Mole(
    atom="""
    O  -1.551007  -0.114520   0.000000
    H  -1.934259   0.762503   0.000000
    H  -0.599677   0.040712   0.000000
    O   1.350625   0.111469   0.000000
    H   1.680398  -0.373741  -0.758561
    H   1.680398  -0.373741   0.758561""",
    basis='aug-cc-pVDZ',
    cart=False, #mol.cart=0 specifices spherical functions and mol.cart=1 specifies Cartesian functions
    charge=0,
    ecp={}, # Effective core potentials (ECP) which replace core electrons around a nucleus by pseudopotentials. It's useful for heavy elements present in a molecule.
    spin=0,
    unit='Angstrom'
)
DWat_mol.build()

### Accéder aux AO integrals

In [ ]:
DWat_mol.intor('int1e_kin')

In [ ]:
DWat_mol.intor('int1e_ipnuc_sph', comp=3)

In [ ]:
DWat_mol.intor('int1e_nuc')

In [ ]:
DWat_mol.intor('int1e_ovlp')

In [ ]:
DWat_mol.intor('int2e')

### Calculs de champ moyen

Une fois l'objet `gto.Mole()` construit, il est nécessaire de créer un objet de champ moyen (mean-field object) Hartree-Fock (HF) ou Kohn-Sham Density Functional Theory (KS-DFT).

Afin de créer un objet champ moyen, il est nécessaire d'importer le sous-module scf (champ auto-cohérent) :

In [ ]:
from pyscf import scf

Pour la plupart des usages, les types de calculs SCF pertinents seront :

1. RHF (Restricted Hartree – Fock)
2. UHF (Hartree-Fock sans restriction)
3. ROHF (Restricted Open-Shell Hartree–Fock)
4. RKS (Kohn-Sham restreint)
5. UKS (Kohn-Sham sans restriction)
6. ROKS (Restricted Open-Shell Kohn – Sham)

L'initialisation de l'objet champ moyen pour ces six types de calculs est illustrée ci-dessous. Ces classes nécessitent au moins un argument, à savoir un objet molécule, mol.

1. `mf=scf.RHF(mol)`
2. `mf=scf.UHF(mol)`
3. `mf=scf.RHF(mol)` (où mol est définie comme étant à couche ouverte (open-shell), mol.spin!=0)
4. `mf=scf.RKS(mol)`
5. `mf=scf.UKS(mol)`
6. `mf=scf.RKS(mol)` (où mol est définie comme open-shell, mol.spin!=0)

#### Création d'un objet mean-field (mf) avec `scf.RHF()`

RHF, ROHF et UHF ont des attributs presque identiques. La première étape consiste à créer l'objet mean-field (RHF dans ce cas) :

In [ ]:
mf=scf.RHF(DWat_mol)

Par la suite, on exécute le caclul avec la méthode `.run()`ou `.kernel()

In [ ]:
mf.run()

In [ ]:
mf.kernel()

Une fois l'exécution du noyau (kernel) SCF achevée, l'objet mean-field (mf) est mis à jour avec plusieurs attributs de sortie utiles :
1. `mf.mo_coeff` - Coefficients d'orbite moléculaire (MO) (matrice où les lignes sont des orbitales atomiques (AO) et les colonnes sont des MO)

In [ ]:
mf.mo_coeff

2. `mf.mo_energy` - énergies MO (vecteur de longueur égale au nombre de MO)

In [ ]:
mf.mo_energy

3. `mf.mo_occ` - Occupation MO (vecteur de longueur égale au nombre de MO)

In [ ]:
mf.mo_occ

4. `mf.e_tot` - Énergie SCF totale en unités de Hartrees

In [ ]:
mf.e_tot

5. `mf.converged` - État de la convergence SCF (True indique convergé et False indique non convergé)

In [ ]:
mf.converged

On peut aussi spécifier les paramètres de calculs du mf.

In [ ]:
mf=scf.RHF(DWat_mol)

mf.conv_tol=1e-12 # the difference in the SCF energy (in Hartrees) between two successive cycles
mf.conv_tol_grad=1e-8 # the root-mean-square of the orbital gradient
mf.direct_scf_tol=1e-13 #the threshold for discarding integrals
mf.init_guess='atom' #  vital for the efficient completion of the SCF procedure
mf.max_cycle=100 # the maximum number of SCF cycles that should be performed before the calculation terminates. For systems that are notoriously difficult to converge, the value should be increased to 100 or even 1000
mf.max_memory=4000 #determines the maximum amount of memory (in Megabytes) that PySCF is allowed to utilize during the SCF procedure
mf.verbose=4 #controls the print level for the mean-field objec

mf.kernel()

#### Création d'un objet mean-field (mf) avec scf.RKS()

La méthode SCF la plus largement utilisée en chimie quantique est sans aucun doute la théorie de la fonctionnelle de la densité (DFT), et exécuter un calcul DFT dans PySCF est presque aussi simple que d'exécuter un calcul Hartree-Fock.

Afin d'accéder à des fonctions supplémentaires qui ne relèvent pas de la norme SCF/HF, il est nécessaire d'importer le module dft (théorie fonctionnelle de la densité) :

In [ ]:
from pyscf import dft

In [ ]:
mf=dft.KS(DWat_mol)

mf.conv_tol=1e-12
mf.conv_tol_grad=1e-8
mf.direct_scf_tol=1e-13
mf.init_guess='atom'
mf.max_cycle=100
mf.max_memory=8000
mf.verbose=0

mf.xc='B3LYP'
mf.grids.atom_grid=(50,194)
mf.grids.becke_scheme=dft.gen_grid.original_becke
mf.grids.prune=None
mf.grids.radi_method=dft.radi.gauss_chebyshev
mf.grids.radii_adjust=None
mf.grids.verbose=4
mf.small_rho_cutoff=1e-10

mf.kernel()

l'energieRHF et RKS (ainsi que UHF et UKS) partagent de nombreux attributs, nous ne couvrirons donc que ceux qui concernent spécifiquement DFT. Ces attributs supplémentaires concernent principalement la fonctionnelle d'échange-corrélation et le maillage utilisé pour son intégration numérique.

#### Attribut xc

(NO DEFAULT)

L'attribut xc définit l'approximation de la fonctionnelle de la densité. Un grand nombre de fonctionnalles de corrélation-échange sont disponibles pour une utilisation dans PySCF, essentiellement toutes celles qui sont disponibles dans la version installée de libxc.

Les règles d'analyse de l'attribut xc soient expliquées dans le [manuel PySCF](http://sunqm.github.io/pyscf/dft.html#customizing-xc-functional). Nous allons passer en revue les principaux exemples pertinents ici :

1. Fonctionnalité de corrélation-échange standard (précodée)

   * Un exemple classique de ce scénario est la très populaire fonction `B3LYP`. Pour exécuter un calcul B3LYP, spécifiez simplement `mf.xc='B3LYP'`.
  
   * Tant que les virgules et les opérateurs ne sont pas utilisés, PySCF supposera que la chaîne indique une fonction de corrélation-échange complète. D'autres exemples peuvent inclure `mf.xc='PBE'`, `mf.xc='TPSS'`, `mf.xc='M06-L'`, etc.

2. Combinaison d'une fonction d'échange unique avec une fonction de corrélation unique

   * Il s'agit d'un choix populaire, en particulier pour l'utilisation de diverses combinaisons des anciennes fonctionnelles d'échange et de corrélation des années 1980 et 1990.
  
   * Dans PySCF, l'insertion d'une virgule dans la chaîne d'attribut xc indique une séparation entre l'échange (à gauche de la virgule) et la corrélation (à droite de la virgule). Par exemple, si on veut exécuter un échange B88 avec une corrélation PBE, on indique `mf.xc='B88,PBE'`.
  
   * Étant donné que la fonctionnelle de corrélation-échange PBE est une combinaison d'échange PBE et de corrélation PBE, il s'ensuit que `mf.xc='PBE'` et `mf.xc='PBE,PBE'` sont des façons exactement équivalentes d'utiliser la fonctionnelle PBE.
  
   * D'autres combinaisons populaires d'une fonction d'échange unique avec une fonction de corrélation unique incluent `mf.xc='B88,LYP'`, `mf.xc='revPBE,PBE'` et `mf.xc='OPTX,LYP' `. Ces combinaisons sont historiquement connues sous le nom de BLYP, revPBE et OLYP, et peuvent être appelées de manière équivalente `mf.xc='BLYP'`, `mf.xc='revPBE'` et `mf.xc='OLYP'`.
  
   * Cette option est particulièrement utile lorsque l'on s'intéresse à une combinaison non conventionnelle, par exemple, la combinaison d'échange PBE avec corrélation TPSS, `mf.xc='PBE,TPSS'`.

3. Fonction d'échange unique ou fonction de corrélation unique

   * L'utilisation d'une fonctionnelle d'échange unique, par exemple, l'échange PBE uniquement, est aussi simple que `mf.xc='PBE,'`. De même, la corrélation PBE ne peut être spécifiée qu'avec `mf.xc=',PBE'`. Dans le premier cas (échange PBE uniquement), le fait qu'il y ait une virgule indique à l'analyseur syntaxique qu'une fonctionnelle xc précodée n'est pas utilisée. Par conséquent, tout ce qui se trouve à gauche de la virgule (PBE) est considéré comme la fonctionnelle d'échange, tandis que tout ce qui se trouve à droite de la virgule (rien) est considéré comme la fonctionnelle de corrélation. La situation inverse s'applique au deuxième cas avec corrélation PBE uniquement.

4. Fonctionnalité de corrélation-échange personnalisée

   * Naturellement, il est possible de personnaliser entièrement l'attribut xc. Encore une fois, tout ce qui se trouve à gauche de la virgule est supposé faire référence à l'échange, et tout ce qui se trouve à droite de la virgule est supposé faire référence à la corrélation.
  
   * Afin de commencer avec quelque chose de simple, nous montrerons comment spécifier la fonctionnelle xc [PBE0](https://aip.scitation.org/doi/abs/10.1063/1.478522) en la construisant à partir de ses composants (la fonctionnelle facile s'écrit simplement `mf.xc='PBE0'`). Le PBE0 comprend 25 % d'échange exact (Hartree-Fock), 75 % d'échange PBE et 100 % de corrélation PBE : `mf.xc='0.25*HF+0.75*PBE,PBE'`.
  
   * Un exemple plus compliqué est celui de B3LYP. B3LYP comprend 20 % d'échange exact (Hartree-Fock), 8 % d'échange LDA, 72 % d'échange B88, 19 % de corrélation VWN et 81 % de corrélation LYP. Malheureusement, il existe 6 paramétrages de la fonctionnelle de corrélation VWN LDA, ce qui a conduit à beaucoup de confusion au fil des ans.
  
   * PySCF, TURBOMOLE, GAMESS, ORCA et PSI4 utilisent tous le paramétrage VWN5, et les deux affectations suivantes sont équivalentes :
     * `mf.xc='B3LYP'`
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN5+0.81*LYP'`
   * Q-Chem et NWChem utilisent le paramétrage VWN1RPA :
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN_RPA+0.81*LYP'`
   * Gaussian utilise la paramétrisation VWN3 :
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN3+0.81*LYP'`    
     
#### nlc attribute

DEFAULT: `mf.nlc=''`

L'attribut nlc est utilisé pour spécifier l'inclusion de la [fonctionnelle de corrélation non locale VV10](https://aip.scitation.org/doi/abs/10.1063/1.3521275). Cela peut simplement être accompli par `mf.nlc='VV10'`. Certaines fonctionnelles de densité telles que [ωB97X-V](http://pubs.rsc.org/en/Content/ArticleLanding/2014/CP/c3cp54374a#!divAbstract), [ωB97M-V](https://aip.scitation .org/doi/abs/10.1063/1.4952647) et [B97M-V](https://aip.scitation.org/doi/abs/10.1063/1.4907719) nécessitent que cet attribut s'exécute correctement.


#### grids attribute/class

Bien que PySCF permette une personnalisation complète de la grille d'intégration DFT, il est conseillé aux utilisateurs non experts de s'appuyer sur les paramètres par défaut car ils ont été affinés pour maximiser la précision et minimiser les coûts de calcul.


##### grids.level attribute

DEFAULT: `mf.grids.level=3`

C'est l'attribut de grille le plus simple à manipuler pour augmenter ou diminuer le nombre de points de grille. Alors que la valeur par défaut est 3, la plage de valeurs possibles commence à `mf.grids.level=0` et continue jusqu'à `mf.grids.level=3`.

#### grids.atom_grid attribute

(NO DEFAULT)

Cet attribut peut être utilisé pour définir le nombre total de couches radiales et le nombre de points de grille angulaire par couche. Le format est (radial, angulaire).

Le nombre de coches radiales peut être défini sur n'importe quel nombre supérieur à 0, tandis que le nombre de points de grille angulaire par couche doit respecter une quadrature de Lebedev valide :

{6,14,26,38,50,74,86,110,146,170,194,230,266,302,350,434,590,770,974,1202,1454,1730,2030,2354,2702,3074,3470,3890,4334,4802,5810}

Les combinaisons couramment utilisées incluent :

* very coarse: `mf.grids.atom_grid=(23,170)`
* coarse: `mf.grids.atom_grid=(50,194)`
* moderate: `mf.grids.atom_grid=(75,302)`
* fine: `mf.grids.atom_grid=(99,590)`
* extremely fine: `mf.grids.atom_grid=(500,974)`

La définition de cet attribut remplacera l'attribut `mf.grids.level`.


#### grids.atomic_radii attribute

DEFAULT: `mf.grids.atomic_radii=dft.radi.BRAGG_RADII`

Cet attribut choisit les valeurs de rayon atomique qui seront utilisées pour ajuster l'espacement des couches radiales. Si `grids.radii_adjust=None`, la définition de ce paramètre n'aura aucun effet. Il existe deux options intégrées dans PySCF :

1. `mf.grids.atomic_radii=dft.radi.BRAGG_RADII`
2. `mf.grids.atomic_radii=dft.radi.COVALENT_RADII`

La première correspond aux valeurs trouvées dans cet article de [J. C. Slater] de 1964 (https://aip.scitation.org/doi/10.1063/1.1725697), tandis que la seconde est principalement construite à partir des valeurs trouvées dans cet [article] ](http://pubs.rsc.org/en/content/articlelanding/2008/dt/b801115j#!divAbstract).

#### grids.becke_scheme attribute

DEFAULT: `mf.grids.becke_scheme=dft.gen_grid.original_becke`

This attribute selects the partition function used to determine the grid point weights. PySCF allows for two options:

1. `mf.grids.becke_scheme=dft.gen_grid.original_becke`
2. `mf.grids.becke_scheme=dft.gen_grid.stratmann`

The first scheme is from the famous 1988 [paper](https://aip.scitation.org/doi/abs/10.1063/1.454033) by Axel Becke, while the second scheme devised by Stratmann, Scuseria, and Frisch is found in this [paper](https://www.sciencedirect.com/science/article/pii/0009261496006008).

Cet attribut sélectionne la fonction de partition utilisée pour déterminer les poids des points de grille. PySCF permet deux options :

1. `mf.grids.becke_scheme=dft.gen_grid.original_becke`
2. `mf.grids.becke_scheme=dft.gen_grid.stratmann`

Le premier schéma est tiré du célèbre article d'[Axel Becke] de 1988 (https://aip.scitation.org/doi/abs/10.1063/1.454033), tandis que le second schéma conçu par [Stratmann, Scuseria et Frisch](https://www.sciencedirect.com/science/article/pii/0009261496006008).

#### grids.prune attribute

DEFAULT: `mf.grids.prune=dft.gen_grid.nwchem_prune`

L'élagage des grilles d'intégration offre l'avantage de réduire le nombre de points de grille (ainsi que le temps de calculs) sans affecter de manière significative l'énergie totale. PySCF a plusieurs options d'élagage intégrées :

1. `mf.grids.prune=dft.gen_grid.nwchem_prune`
2. `mf.grids.prune=dft.gen_grid.sg1_prune`
3. `mf.grids.prune=dft.gen_grid.treutler_prune`
4. `mf.grids.prune=None`

#### grids.radi_method attribute

DEFAULT: `mf.grids.radi_method=dft.radi.treutler_ahlrichs`

Implémentation des couches radiales.

1. `mf.grids.radi_method=dft.radi.treutler_ahlrichs`
2. `mf.grids.radi_method=dft.radi.delley`
3. `mf.grids.radi_method=dft.radi.mura_knowles`
4. `mf.grids.radi_method=dft.radi.gauss_chebyshev`

#### grids.radii_adjust attribute

DEFAULT: `mf.grids.radii_adjust=dft.radi.treutler_atomic_radii_adjust`

Il existe deux options pour ajuster les rayons atomiques.

1. `mf.grids.radii_adjust=dft.radi.treutler_atomic_radii_adjust`
2. `mf.grids.radii_adjust=dft.radi.becke_atomic_radii_adjust`
3. `mf.grids.radii_adjust=None`

#### grids.verbose attribute

DEFAULT: `mf.grids.verbose=0`

L'attribut verbose contrôle le niveau d'impression de la grille d'intégration. Le réglage `mf.grids.verbose=0` n'imprimera aucune information sur la grille, tandis que le réglage `mf.grids.verbose=4` imprime des informations utiles telles que le nombre total de points de grille, ainsi que des informations concernant les schémas utilisés pour implémenter la grille.

#### nlcgrids attribute/class

L'attribut `nlcgrids` fonctionne de la même manière que l'attribut `grids`. La seule différence est que `nlcgrids` contrôle les paramètres de grille pour la fonctionnlle de corrélation non locale, tandis que `grids` contrôle la grille pour la fonctionnelle de corrélation-échange locale. La fonctionnelle de corrélation non locale VV10 nécessite une grille beaucoup plus grossière que nécessaire pour la fonctionnelle xc locale, ce qui est une bonne chose car la mise à l'échelle de nlc est quadratique en points de grille (boucles à travers r et r').

#### small_rho_cutoff attribute

DEFAULT: `mf.small_rho_cutoff=1e-07`

Cet attribut est utilisé pour éliminer les points de grille qui contribuent de manière négligeable au nombre total d'électrons tel que calculé par la grille d'intégration choisie. Étant donné un vecteur, **r**, de longueur ng (nombre de points de grille), qui contient la valeur de la densité électronique à chaque point de grille, et un vecteur, **w**, de longueur ng qui contient les poids associés à chaque point de grille, les points de grille sont ignorés si **r**∘**w** ≤ `mf.small_rho_cutoff`.


![DFT_Flowchart.jpg](./Graphics/DFT_Flowchart.jpg)

Organigramme conceptuel de la prise de décision en étapes élémentaires dans les calculs typiques de chimie computationnelle moléculaire.

In [3]:
import qiskit.tools.jupyter

%qiskit_version_table

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)